In [47]:
import numpy as np
import sklearn.neural_network
import pandas as pd

In [3]:
f1 = open('data/profiles.csv', 'r')
f2 = open('data/profiles_fixed.csv', 'w')
for line in f1:
    line = line.replace('Micronesia, Federated States of', 'Federated States of Micronesia')
    line = line.replace('Korea, Republic of', 'Republic of Korea')
    line = line.replace('Iran, Islamic Republic of', 'Islamic Republic of Iran')
    line = line.replace('Virgin Islands, British', 'British Virgin Islands')
    line = line.replace('Korea, Democratic People\'s Republic of', 'Democratic People\'s Republic of Korea')
    line = line.replace('Palestinian Territory, Occupied', 'Palestinian Territory Occupied')
    line = line.replace('Tanzania, United Republic of', 'Tanzania United Republic of')
    line = line.replace('Congo, the Democratic Republic of the', 'Congo the Democratic Republic of the')
    line = line.replace('Virgin Islands, U.s.', 'Virgin Islands U.s.')
    line = line.replace('Virgin Islands, U.s.', 'Virgin Islands U.s.')
    f2.write(line)
f1.close()
f2.close()

In [4]:
# relies on chopping off column headers ahead of time. 
artists = np.loadtxt('data/artists.csv', dtype='string', delimiter=',')
print artists
profiles = np.loadtxt('data/profiles_fixed.csv', dtype='string', delimiter=',')
print profiles.shape

# fixing erroneous, irrelevant ages like 1002
max_age = 120
for p in profiles:
    if p[2] != '' and (int(p[2]) >= max_age or int(p[2]) < 0):
        p[2] = ''

[['03098741-08b3-4dd7-b3f6-1b0bfa2c879c' 'Liars']
 ['69c4cc43-8163-41c5-ac81-30946d27bb69' 'CunninLynguists']
 ['7a2e6b55-f149-4e74-be6a-30a1b1a387bb' 'The Desert Sessions']
 ..., 
 ['8974da95-e631-45aa-8fd7-aa0c2795f997' 'Harry Gregson-Williams']
 ['8067c102-4996-42bc-9980-06ce2e644eae' 'Soul Coughing']
 ['39c2a93d-9afa-4a22-9bba-c087ab056e1c' 'Jefferson Airplane']]
(233286, 4)


In [5]:
nation_dict = {}
count = 0
for p in profiles:
    if p[3] not in nation_dict:
        nation_dict[p[3]] = None
num_nations = len(nation_dict)
count = 0
for key in nation_dict.keys():
    nation_dict[key] = np.zeros(num_nations)
    nation_dict[key][count] = 1
    count += 1

In [6]:
train = np.loadtxt('data/train.csv', dtype='string', delimiter=',')
print train

[['eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03'
  '5a8e07d5-d932-4484-a7f7-e700793a9c94' '554']
 ['44ce793a6cd9d20f13f4a576a818ef983314bb5d'
  'a3a92047-be1c-4f3e-8960-c4f8570984df' '81']
 ['da9cf3f557161d54b76f24db64be9cc76db008e3'
  'eeb1195b-f213-4ce1-b28c-8565211f8e43' '708']
 ..., 
 ['b464ce35f8f875fdf686fd3dc4ce165b650bde7a'
  '199d0ab3-22c0-4c00-b99d-257f591dda1f' '29']
 ['d8ccd698ad712c26d2620dff07f27473b9173ad9'
  'a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432' '56']
 ['b7d94e5f82e157f94df6c93f283a181108f780c4'
  '7527f6c2-d762-4b88-b5e2-9244f1e34c46' '240']]


In [7]:
artists_1h = {}
num_art = len(artists)
for a in range(num_art):
    one_hot = np.zeros(num_art)
    one_hot[a] = 1
    artists_1h[artists[a][0]] = one_hot

In [8]:
profiles_1h = {}
num_prof = len(profiles)
for p in range(num_prof):
    
#     one_hot_id = np.zeros(num_prof)
#     one_hot_id[p] = 1
    
    one_hot_gender = np.zeros(3)
    if profiles[p][1] == 'f':
        one_hot_gender[0] = 1
    elif profiles[p][1] == 'm':
        one_hot_gender[1] = 1
    else:
        one_hot_gender[2] = 1

    one_hot_age = np.zeros(max_age + 1)
    if profiles[p][2] == '':
        one_hot_age[max_age] = 1
    else:
        one_hot_age[int(profiles[p][2])] = 1
        
    one_hot_nation = nation_dict[profiles[p][3]]
    
#     profiles_1h[profiles[p][0]] = np.concatenate((one_hot_id, one_hot_gender, one_hot_age, one_hot_nation))
    profiles_1h[profiles[p][0]] = np.concatenate((one_hot_gender, one_hot_age, one_hot_nation))

In [44]:
# temp = np.concatenate((profiles_1h[train[0][0]], artists_1h[train[0][1]]))
# print temp
# X = np.array(temp)
# Y = np.array(train[0][1])
X = np.array(np.zeros(2366).T)
X = np.array([X])
Y = np.array([])
for t in train[:1000]:
    temp = np.concatenate((profiles_1h[t[0]], artists_1h[t[1]]))
#     print X.shape
#     print temp.shape
    X = np.append(X, [temp], axis=0)
    Y = np.append(Y, int(t[2]))
X = X[1:]
print X.shape
print Y.shape

(1000, 2366)
(1000,)


In [45]:
X_cv = np.array(np.zeros(2366).T)
X_cv = np.array([X_cv])
Y_cv = np.array([])
for t in train[:1000]:
    temp = np.concatenate((profiles_1h[t[0]], artists_1h[t[1]]))
#     print X.shape
#     print temp.shape
    X_cv = np.append(X_cv, [temp], axis=0)
    Y_cv = np.append(Y_cv, int(t[2]))
X_cv = X_cv[1:]

In [49]:
NN = sklearn.neural_network.MLPRegressor()
NN.fit(X, Y)
preds = NN.predict(X_cv)
error = preds - Y_cv
print preds
print Y_cv

[ 302.54173565  190.26622753  275.34414134  265.75191003  255.10068033
  299.16108934  267.14130193  266.16896829  284.61931768  232.23632153
  184.9167149   150.5295819   267.10630083  153.81496251  188.29491306
  221.45789989  194.876341    151.40820511  213.62836498  277.11748577
  212.34396671  187.91052275  293.56197089  272.44836038  234.10830551
  255.31017482  253.7482885   178.5687672   195.62284855   79.03222695
  211.37986873  203.07108411  276.28816296  295.22324002  252.159771
  279.00777952  211.40505939  298.11151093  242.18371979  277.92885264
  182.70741398  192.42552712  157.0939037   297.57696199  122.49714003
  228.422835     72.15054693  236.50420873  171.76516621  237.3593162
  263.92768195  179.03371959  286.98267848  270.85060326  228.67850251
  262.92704848  230.72716252   50.30614953  249.20585602  289.48032844
  188.71380289  231.39972329  165.62771657  179.38132775  282.54239211
  285.03970041  286.94134481  216.91158047  210.49272819  214.28411784
  106.811